In [76]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from pathlib import Path
import awkward as ak
import hdf5plugin

In [77]:
datapath = Path("/disk/atlas3/data_MC/topTagging/top")

In [78]:
def load_array(path:Path):
    with h5py.File(path) as f:
        labels = ["event_id", "clusters", "truth_jet", "ttv", "is_signal_new"]
        data = f["table"]["table"] #dataset, each row is a void type
        print(data.dtype)
        eventid, clusters, misc = data['index'], data['values_block_0'], data['values_block_1']
        truthjets = clusters[:,-4:]
        clusters = clusters[:,:-4]
        clusters = np.transpose(clusters.reshape(len(clusters), 200, 4), (2, 0, 1))
        
        print(clusters.shape)
        cluster_labels = ["E", "x", "y", "z"]
        cluster_arr = ak.Array(dict(zip(cluster_labels, clusters)))
        
        arr = ak.Array(dict(zip(labels, [eventid, cluster_arr, truthjets, misc[:,0], misc[:,1]])))
        return arr

In [79]:
train = load_array(datapath/'train.h5')

[('index', '<i8'), ('values_block_0', '<f4', (804,)), ('values_block_1', '<i8', (2,))]
(4, 1211000, 200)


In [80]:
def zero_to_nan(arr):
    clusters = arr.clusters
    clusters.E = ak.where(clusters.E == 0, np.nan, clusters.E)
    clusters.x = ak.where(clusters.x == 0, np.nan, clusters.x)
    clusters.y = ak.where(clusters.y == 0, np.nan, clusters.y)
    clusters.z = ak.where(clusters.z == 0, np.nan, clusters.z)
    arr.clusters = clusters
    return arr



In [81]:
def cartesian_to_angles(arr, drop:bool=False):
    clusters = arr.clusters
    clusters["pT"] = np.sqrt(clusters.x**2 + clusters.y**2)
    clusters["eta"] = np.arcsinh(clusters.z/clusters["pT"])
    clusters["phi"] = np.arcsin(clusters.y/clusters["pT"])
    """
    Add this functionality
    if drop:
    """
    arr.clusters = clusters
    return arr



In [82]:
testing = train[0:20]
testing = zero_to_nan(testing)
testing = cartesian_to_angles(testing)

[{E: [474, 103, 105, 40.2, 22.4, 20.3, 19, 13.5, 11.2, ... 0, 0, 0, 0, 0, 0, 0, 0]}]
